In [1]:
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np



In [3]:
stavbe2020 = pd.read_csv(r"clean_GURS\new_prodaja2020.csv", sep=',', encoding='utf-8')
stavbe2020.head(10)

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja
0,KAŠELJ,LJUBLJANA,1978,86,86,2020,125000,NaN
1,BEŽIGRAD,LJUBLJANA,1960,36,29,2020,91818,NaN
2,BRINJE I,LJUBLJANA,1963,69,69,2020,140000,NaN
3,KARLOVŠKO PREDMESTJE,LJUBLJANA,1999,147,147,2020,405000,NaN
4,VIČ,LJUBLJANA,2009,144,136,2020,370000,NaN
5,VIČ,LJUBLJANA,2008,80,61,2020,195000,NaN
6,SPODNJA ŠIŠKA,LJUBLJANA,1908,112,82,2020,142000,NaN
7,ŠENTVID NAD LJUBLJANO,LJUBLJANA,1965,72,47,2020,128000,NaN
8,POLJANSKO PREDMESTJE,LJUBLJANA,1977,109,98,2020,364000,NaN
9,TRNOVSKO PREDMESTJE,LJUBLJANA,1981,80,74,2020,241000,NaN


In [4]:
stavbe2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860 entries, 0 to 2859
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ime_ko             2860 non-null   object 
 1   naselje            2842 non-null   object 
 2   letnica_izgradnje  2860 non-null   int64  
 3   m2_bruto           2860 non-null   int64  
 4   m2_neto            2860 non-null   int64  
 5   LETO               2860 non-null   int64  
 6   cena               2860 non-null   int64  
 7   novogradnja        516 non-null    float64
dtypes: float64(1), int64(5), object(2)
memory usage: 178.9+ KB


In [5]:
#preparing the year column for Power BI
stavbe2020['LETO'] = pd.to_datetime("2020-01-01")

In [6]:

# creating a boolean column 'NOVOGRADNJA_1' based on 'NOVOGRADNJA'
stavbe2020['novogradnja_1'] = stavbe2020['novogradnja'] == 1

# Preview the new column
stavbe2020[['novogradnja', 'novogradnja_1']].head(10)


,novogradnja,novogradnja_1
0,NaN,False
1,NaN,False
2,NaN,False
3,NaN,False
4,NaN,False
5,NaN,False
6,NaN,False
7,NaN,False
8,NaN,False
9,NaN,False


In [7]:
#Printing the number of rows with missing values in 'm2_neto' column
stavbe2020[stavbe2020['m2_neto'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja,novogradnja_1


In [8]:
def oceni_sobnost(m2):
    if pd.isna(m2):
        return None
    if m2 < 35:
        return 1
    elif m2 < 50:
        return 2
    elif m2 < 70:
        return 3
    elif m2 < 90:
        return 4
    elif m2 < 110:
        return 5
    else:
        return 6

In [9]:
stavbe2020['sobnost_ocenjena'] = stavbe2020['m2_neto'].apply(oceni_sobnost)

In [10]:
stavbe2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860 entries, 0 to 2859
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             2860 non-null   object        
 1   naselje            2842 non-null   object        
 2   letnica_izgradnje  2860 non-null   int64         
 3   m2_bruto           2860 non-null   int64         
 4   m2_neto            2860 non-null   int64         
 5   LETO               2860 non-null   datetime64[ns]
 6   cena               2860 non-null   int64         
 7   novogradnja        516 non-null    float64       
 8   novogradnja_1      2860 non-null   bool          
 9   sobnost_ocenjena   2860 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 204.0+ KB


In [11]:
# Droping the original 'NOVOGRADNJA' column, as we now have 'NOVOGRADNJA_1' which has the same information in a boolean format.
stavbe2020.drop(columns=['novogradnja'], inplace=True)

In [12]:
#Printing the number of rows with missing values in 'm2_neto' column
stavbe2020[stavbe2020['m2_neto'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja_1,sobnost_ocenjena


In [13]:
stavbe2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860 entries, 0 to 2859
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             2860 non-null   object        
 1   naselje            2842 non-null   object        
 2   letnica_izgradnje  2860 non-null   int64         
 3   m2_bruto           2860 non-null   int64         
 4   m2_neto            2860 non-null   int64         
 5   LETO               2860 non-null   datetime64[ns]
 6   cena               2860 non-null   int64         
 7   novogradnja_1      2860 non-null   bool          
 8   sobnost_ocenjena   2860 non-null   int64         
dtypes: bool(1), datetime64[ns](1), int64(5), object(2)
memory usage: 181.7+ KB


In [14]:
# Uniforming the column names to lowercase
stavbe2020.columns = [col.lower() for col in stavbe2020.columns]

In [15]:
#uniforming the all values in all the columns to lowercase
for col in stavbe2020.columns:
    if stavbe2020[col].dtype == 'object':
        stavbe2020[col] = stavbe2020[col].str.lower()

In [16]:
stavbe2020['ime_ko'].unique()

array(['kašelj', 'bežigrad', 'brinje i', 'karlovško predmestje', 'vič',
       'spodnja šiška', 'šentvid nad ljubljano', 'poljansko predmestje',
       'trnovsko predmestje', 'dravlje', 'moste', 'slape', 'štepanja vas',
       'stožice', 'tabor', 'črnuče', 'zelena jama', 'zgornja šiška',
       'šentpeter', 'dobrova', 'brinje ii', 'gradišče ii', 'gradišče i',
       'ljubljana mesto', 'šmartno pod šmarno goro', 'dobrunje', 'rudnik',
       'udmat', 'zadobrova', 'ajdovščina', 'brdo', 'ježica', 'prule',
       'glince', 'vižmarje', 'podmolnik', 'nove jarše',
       'krakovsko predmestje', 'bizovik', 'tacen', 'podsmreka', 'javor',
       'golovec', 'trebeljevo', 'nadgorica', 'šmartno ob savi',
       'gameljne', 'šujica', 'stanežiče', 'iška loka', 'lipoglav',
       'sostro', 'volavlje', 'grič', 'podgorica', 'tomišelj'],
      dtype=object)

In [17]:
# checking all the empty values in the 'naselje' column
stavbe2020[stavbe2020['naselje'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,leto,cena,novogradnja_1,sobnost_ocenjena
56,šentvid nad ljubljano,NaN,2019,166,127,2020-01-01,200000,False,6
66,kašelj,NaN,2018,162,0,2020-01-01,231000,True,1
586,kašelj,NaN,2018,147,0,2020-01-01,240000,True,1
594,šmartno ob savi,NaN,2019,126,126,2020-01-01,270000,False,6
689,zadobrova,NaN,2019,147,0,2020-01-01,300000,False,1
840,zadobrova,NaN,2019,146,0,2020-01-01,300000,False,1
1217,lipoglav,NaN,1995,82,65,2020-01-01,150000,False,3
1274,šmartno ob savi,NaN,2019,126,126,2020-01-01,290000,False,6
1418,volavlje,NaN,1970,72,56,2020-01-01,29000,False,3
1453,poljansko predmestje,NaN,1895,64,39,2020-01-01,81500,False,2


In [18]:
# changing all the empty values in the 'naselje' column to 'ljubljana'
stavbe2020['naselje'].fillna('ljubljana', inplace=True)

C:\Users\gaj\AppData\Local\Temp\ipykernel_15444\1464791112.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stavbe2020['naselje'].fillna('ljubljana', inplace=True)


In [19]:
stavbe2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2860 entries, 0 to 2859
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             2860 non-null   object        
 1   naselje            2860 non-null   object        
 2   letnica_izgradnje  2860 non-null   int64         
 3   m2_bruto           2860 non-null   int64         
 4   m2_neto            2860 non-null   int64         
 5   leto               2860 non-null   datetime64[ns]
 6   cena               2860 non-null   int64         
 7   novogradnja_1      2860 non-null   bool          
 8   sobnost_ocenjena   2860 non-null   int64         
dtypes: bool(1), datetime64[ns](1), int64(5), object(2)
memory usage: 181.7+ KB


In [20]:
stavbe2020.to_csv(r"clean_GURS\stavbe2020_cleaned.csv", index=False, encoding='utf-8')